In [2]:
def distribute_segment(
    start_level: int,   # 段起始等级，如 A 段：1
    end_level: int,     # 段结束等级，如 A 段：10
    total_time: float,  # 该段总用时(分钟)
    total_exp: float,   # 该段总经验
    ratio: float,       # 段内的指数比率(如1.1,1.2等)
    base_time: float,   # 进入本段前已累计的时间(分钟)
    base_exp: float,    # 进入本段前已累计的经验
    stage_name: str,    # 场次名称: A/B/C/D
    exp_per_min: float  # 该段的经验效率(经验/分钟)，用于信息展示
):
    """
    对从start_level+1级到end_level级(共 end_level - start_level 次升级)做指数分配。
    返回一个list，list中每个元素是一个dict，包含：level, minute, exp, stage, exp_per_min。
    """
    results = []
    
    # 要经历的“升级次数”
    num_upgrades = end_level - start_level
    if num_upgrades <= 0:
        return results
    
    # 计算几何级数之和 S = 1 + r + r^2 + ... + r^(n-1)
    # 这里用循环累加，或用公式 S = (r^n - 1)/(r - 1)
    S = 0.0
    for k in range(num_upgrades):
        S += ratio ** k
    
    current_time = base_time
    current_exp  = base_exp
    
    # 逐级计算
    for i in range(1, num_upgrades + 1):
        # i: 第 i 次升级
        # 升级后等级:
        new_level = start_level + i
        
        # 权重比例 = ratio^(i-1)
        weight = ratio ** (i - 1)
        
        # 本次升级所消耗的时间
        delta_time = total_time * (weight / S)
        # 本次升级所需要的经验
        delta_exp  = total_exp  * (weight / S)
        
        current_time += delta_time
        current_exp  += delta_exp
        
        # 记录本次升级完成后(即到达 new_level)时的信息
        row = {
            "level": new_level,
            "minute": current_time,
            "exp": current_exp,
            "stage": stage_name,
            "exp_per_min": exp_per_min
        }
        results.append(row)
    
    return results


def main():
    # 1. 定义各段的参数（仅供示例，可根据实际需求调整）
    # -------------------------
    # A 段 (1~10级)
    A_segment_time = 60.0
    A_segment_exp  = 60.0 * 221.4   # ~13,284
    A_ratio        = 1.2
    
    # B 段 (10~25级)
    B_segment_time = 600.0
    B_segment_exp  = 600.0 * 635.6  # 381,360
    B_ratio        = 1.1
    
    # C 段 (25~45级)
    C_segment_time = 1800.0
    C_segment_exp  = 1800.0 * 1950  # 3,510,000
    C_ratio        = 1.06
    
    # D 段 (45~60级)
    D_segment_time = 4860.0
    D_segment_exp  = 4860.0 * 4068.2  # ~19,771,172
    D_ratio        = 1.1
    
    # 2. 逐段调用 distribute_segment
    # -------------------------
    all_results = []
    
    # A 段起始：1级，时间=0，经验=0
    # 先手动记录：初始状态(1级)
    initial_row = {
        "level": 1,
        "minute": 0.0,
        "exp": 0.0,
        "stage": "A",
        "exp_per_min": 221.4
    }
    all_results.append(initial_row)
    
    # (1) A 段：1 --> 10
    A_results = distribute_segment(
        start_level   = 1,
        end_level     = 10,
        total_time    = A_segment_time,
        total_exp     = A_segment_exp,
        ratio         = A_ratio,
        base_time     = 0.0,       # 上段结束的时间
        base_exp      = 0.0,       # 上段结束的经验
        stage_name    = "A",
        exp_per_min   = 221.4
    )
    all_results.extend(A_results)
    
    # 记录 A 段结束时的时间/经验，作为 B 段起点
    A_end_time = A_results[-1]["minute"]
    A_end_exp  = A_results[-1]["exp"]
    
    # (2) B 段：10 --> 25
    B_results = distribute_segment(
        start_level   = 10,
        end_level     = 25,
        total_time    = B_segment_time,
        total_exp     = B_segment_exp,
        ratio         = B_ratio,
        base_time     = A_end_time,
        base_exp      = A_end_exp,
        stage_name    = "B",
        exp_per_min   = 635.6
    )
    all_results.extend(B_results)
    
    B_end_time = B_results[-1]["minute"]
    B_end_exp  = B_results[-1]["exp"]
    
    # (3) C 段：25 --> 45
    C_results = distribute_segment(
        start_level   = 25,
        end_level     = 45,
        total_time    = C_segment_time,
        total_exp     = C_segment_exp,
        ratio         = C_ratio,
        base_time     = B_end_time,
        base_exp      = B_end_exp,
        stage_name    = "C",
        exp_per_min   = 1950.0
    )
    all_results.extend(C_results)
    
    C_end_time = C_results[-1]["minute"]
    C_end_exp  = C_results[-1]["exp"]
    
    # (4) D 段：45 --> 60
    D_results = distribute_segment(
        start_level   = 45,
        end_level     = 60,
        total_time    = D_segment_time,
        total_exp     = D_segment_exp,
        ratio         = D_ratio,
        base_time     = C_end_time,
        base_exp      = C_end_exp,
        stage_name    = "D",
        exp_per_min   = 4068.2
    )
    all_results.extend(D_results)
    
    # 3. 在这里，all_results 就包含了 1~60 每级的信息
    # -------------------------
    return all_results


if __name__ == "__main__":
    results = main()
    
    # 简单打印前 10 行和最后 5 行做演示
    print("=== 前 10 行 ===")
    for row in results[:10]:
        print(row)
    
    print("\n=== 最后 5 行 ===")
    for row in results[-5:]:
        print(row)
    
    # 如需全部数据，可自行打印/写入CSV/Excel
    # 例如：
    import csv
    with open("level_data.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["level", "minute", "exp", "stage", "exp_per_min"])
        writer.writeheader()
        writer.writerows(results)


=== 前 10 行 ===
{'level': 1, 'minute': 0.0, 'exp': 0.0, 'stage': 'A', 'exp_per_min': 221.4}
{'level': 2, 'minute': 2.8847677003499634, 'exp': 638.6875688574819, 'stage': 'A', 'exp_per_min': 221.4}
{'level': 3, 'minute': 6.34648894076992, 'exp': 1405.1126514864602, 'stage': 'A', 'exp_per_min': 221.4}
{'level': 4, 'minute': 10.500554429273867, 'exp': 2324.822750641234, 'stage': 'A', 'exp_per_min': 221.4}
{'level': 5, 'minute': 15.485433015478604, 'exp': 3428.4748696269626, 'stage': 'A', 'exp_per_min': 221.4}
{'level': 6, 'minute': 21.46728731892429, 'exp': 4752.857412409837, 'stage': 'A', 'exp_per_min': 221.4}
{'level': 7, 'minute': 28.645512483059107, 'exp': 6342.116463749286, 'stage': 'A', 'exp_per_min': 221.4}
{'level': 8, 'minute': 37.25938268002089, 'exp': 8249.227325356625, 'stage': 'A', 'exp_per_min': 221.4}
{'level': 9, 'minute': 47.59602691637503, 'exp': 10537.760359285432, 'stage': 'A', 'exp_per_min': 221.4}
{'level': 10, 'minute': 60.0, 'exp': 13284.0, 'stage': 'A', 'exp_per_mi

In [4]:
import pandas as pd

def distribute_segment(
    start_level: int,
    end_level: int,
    total_time: float,
    total_exp: float,
    ratio: float,
    base_time: float,
    base_exp: float,
    stage_name: str,
    exp_per_min: float
):
    """
    对 [start_level+1, end_level] 做指数分配，返回一个list(dict)：
    {
      "level": 等级,
      "minute": 当前累计分钟(浮点),
      "exp": 当前累计经验(浮点, 原始值),
      "stage": 场次,
      "exp_per_min": 经验效率(浮点)
    }
    """
    results = []
    
    num_upgrades = end_level - start_level
    if num_upgrades <= 0:
        return results
    
    # 计算几何级数之和
    S = 0.0
    for k in range(num_upgrades):
        S += ratio ** k
    
    current_time = base_time
    current_exp  = base_exp
    
    for i in range(1, num_upgrades + 1):
        weight = ratio ** (i - 1)
        delta_time = total_time * (weight / S)
        delta_exp  = total_exp  * (weight / S)
        
        current_time += delta_time
        current_exp  += delta_exp
        
        results.append({
            "level": start_level + i,
            "minute": current_time,
            "exp": current_exp,  # 暂时保留原始浮点
            "stage": stage_name,
            "exp_per_min": exp_per_min
        })
    
    return results


def main():
    # --- 各段参数 ---
    # A 段 (1~10)
    A_segment_time = 60.0
    A_segment_exp  = 60.0 * 221.4  # ~13,284
    A_ratio        = 1.2
    
    # B 段 (10~25)
    B_segment_time = 600.0
    B_segment_exp  = 600.0 * 635.6  # 381,360
    B_ratio        = 1.1
    
    # C 段 (25~45)
    C_segment_time = 1800.0
    C_segment_exp  = 1800.0 * 1950  # 3,510,000
    C_ratio        = 1.06
    
    # D 段 (45~60)
    D_segment_time = 4860.0
    D_segment_exp  = 4860.0 * 4068.2  # ~19,771,172
    D_ratio        = 1.1
    
    # --- 汇总存储 ---
    all_records = []
    
    # 初始行: 1级, time=0, exp=0
    # 这里可以先手动插入一行 1级数据
    all_records.append({
        "level": 1,
        "minute": 0.0,
        "exp": 0.0,
        "stage": "A",
        "exp_per_min": 221.4
    })
    
    # A 段
    A_res = distribute_segment(
        start_level   = 1,
        end_level     = 10,
        total_time    = A_segment_time,
        total_exp     = A_segment_exp,
        ratio         = A_ratio,
        base_time     = 0.0,
        base_exp      = 0.0,
        stage_name    = "A",
        exp_per_min   = 221.4
    )
    all_records.extend(A_res)
    
    A_end_time = A_res[-1]["minute"]
    A_end_exp  = A_res[-1]["exp"]
    
    # B 段
    B_res = distribute_segment(
        start_level   = 10,
        end_level     = 25,
        total_time    = B_segment_time,
        total_exp     = B_segment_exp,
        ratio         = B_ratio,
        base_time     = A_end_time,
        base_exp      = A_end_exp,
        stage_name    = "B",
        exp_per_min   = 635.6
    )
    all_records.extend(B_res)
    
    B_end_time = B_res[-1]["minute"]
    B_end_exp  = B_res[-1]["exp"]
    
    # C 段
    C_res = distribute_segment(
        start_level   = 25,
        end_level     = 45,
        total_time    = C_segment_time,
        total_exp     = C_segment_exp,
        ratio         = C_ratio,
        base_time     = B_end_time,
        base_exp      = B_end_exp,
        stage_name    = "C",
        exp_per_min   = 1950.0
    )
    all_records.extend(C_res)
    
    C_end_time = C_res[-1]["minute"]
    C_end_exp  = C_res[-1]["exp"]
    
    # D 段
    D_res = distribute_segment(
        start_level   = 45,
        end_level     = 60,
        total_time    = D_segment_time,
        total_exp     = D_segment_exp,
        ratio         = D_ratio,
        base_time     = C_end_time,
        base_exp      = C_end_exp,
        stage_name    = "D",
        exp_per_min   = 4068.2
    )
    all_records.extend(D_res)
    
    return all_records

def human_readable_int(value: float) -> str:
    """
    先四舍五入为整数，然后对大数进行 K/M/B/T 缩写。
    """
    int_val = round(value)
    if int_val == 0:
        return "0"
    
    s = str(int_val)
    length = len(s)
    
    suffixes = ["", "K", "M", "B", "T"]
    group_index = (length - 1) // 3
    if group_index >= len(suffixes):
        group_index = len(suffixes) - 1
    
    suffix = suffixes[group_index]
    keep_digits_count = length - group_index * 3
    if keep_digits_count < 1:
        keep_digits_count = 1
    
    main_part = s[:keep_digits_count]
    formatted = main_part
    if group_index > 0:
        formatted += suffix
    
    return formatted

def export_to_excel():
    import pandas as pd
    
    # 1. 获取数据
    records = main()
    
    # 2. 转换成 DataFrame
    df = pd.DataFrame(records)
    
    # 3. 格式化 exp
    #    - 没有小数部分
    #    - 大数缩写
    df["exp"] = df["exp"].apply(human_readable_int)
    
    # 4. minute(时间) 若想去掉小数部分，也可类似处理，这里保留一位小数示范
    #   也可改成整数 df["minute"] = df["minute"].round(0).astype(int)
    df["minute"] = df["minute"].round(1)
    
    # 5. exp_per_min 看需求留多少位小数，这里保留1位小数
    df["exp_per_min"] = df["exp_per_min"].round(1)
    
    # 6. 输出到 Excel
    df.to_excel("level_data.xlsx", index=False)
    
    print("数据已保存到 level_data.xlsx")

if __name__ == "__main__":
    export_to_excel()



数据已保存到 level_data.xlsx


In [5]:
import math

def stamina_raw(x):
    # a, b, c 取上面求解得到的近似值
    a = -3924
    b = 4275
    c = 1.05773
    return a + b * (c ** x)

# 生成表: Level -> Stamina
for level in range(1, 61):
    val = stamina_raw(level)
    print(level, val)

def stamina_final(x):
    raw_val = stamina_raw(x)
    # 四舍五入到个位数
    #   final_val = round(raw_val)
    # 如果想让尾数更好看(保留到10)
    final_val = int(round(raw_val, -1))  # -1 表示保留到十位
    return final_val

# 测试几级
for lvl in [1, 11, 60]:
    print(lvl, stamina_raw(lvl), "->", stamina_final(lvl))

# put in dataframe
import pandas as pd

levels = list(range(1, 61))
raw_stamina = [stamina_raw(lvl) for lvl in levels]
final_stamina = [stamina_final(lvl) for lvl in levels]

df = pd.DataFrame({
    "Level": levels,
    "Raw Stamina": raw_stamina,
    "Final Stamina": final_stamina
})

df.to_excel("stamina_data.xlsx", index=False)
df

1 597.7957500000002
2 858.8390186475008
3 1134.9523151940211
4 1427.0056323501722
5 1735.9191875057477
6 2062.6663222004554
7 2408.2765689810876
8 2773.838895308367
9 3160.505134734519
10 3569.493616162743
11 4002.0930026238184
12 4459.666351665292
13 4943.655410146928
14 5455.585156974712
15 5997.068608086864
16 6569.81189883172
17 7175.619659751275
18 7816.4007027087155
19 8494.17403527609
20 9211.07522233258
21 9969.36311491784
22 10771.426967542047
23 11619.79396637825
24 12517.137192057271
25 13466.284042154737
26 14470.225139908329
27 15532.12375723524
28 16655.325781740434
29 17843.370259120307
30 19100.000544179326
31 20429.176095594798
32 21835.084951593486
33 23322.15692584898
34 24895.077565178242
35 26558.802913015985
36 28318.575125184398
37 30179.938987161295
38 32148.759384890123
39 34231.23978417983
40 36433.94177692053
41 38763.80575570215
42 41228.17278197884
43 43834.80771668249
44 46591.92368616657
45 49508.20796056897
46 52592.849326132615
47 55855.56703773026
48 5

,Level,Raw Stamina,Final Stamina
0,1,597.795750,600
1,2,858.839019,860
2,3,1134.952315,1130
3,4,1427.005632,1430
4,5,1735.919188,1740
5,6,2062.666322,2060
6,7,2408.276569,2410
7,8,2773.838895,2770
8,9,3160.505135,3160
9,10,3569.493616,3570


In [6]:
import math

# 拟合得到的参数
a = -4028.35
b = 4766
c = 1.01308

def stamina_raw(level):
    return a + b * (c ** level)

# 打印示例
for lvl in range(1, 61):
    raw_val = stamina_raw(lvl)
    print(f"Level={lvl}, StaminaRaw={raw_val:.2f}")


def stamina_final(level):
    val = stamina_raw(level)
    # 先四舍五入到整数
    # val_rounded = round(val)

    # 或者：四舍五入到最近的 10
    val_rounded_10 = round(val, -1)  # -1 表示保留到十位
    return int(val_rounded_10)

# 测试打印关键等级
print("Lv 1  =>", stamina_final(1))   # ~800
print("Lv 10 =>", stamina_final(10))  # ~1400
print("Lv 60 =>", stamina_final(60))  # ~6340

# 生成表格
import pandas as pd

levels = list(range(1, 61))
raw_stamina = [stamina_raw(lvl) for lvl in levels]
final_stamina = [stamina_final(lvl) for lvl in levels]

df = pd.DataFrame({
    "Level": levels,
    "Raw Stamina": raw_stamina,
    "Final Stamina": final_stamina
})

df.to_excel("stamina_data_v3.xlsx", index=False)
df

Level=1, StaminaRaw=799.99
Level=2, StaminaRaw=863.14
Level=3, StaminaRaw=927.12
Level=4, StaminaRaw=991.94
Level=5, StaminaRaw=1057.61
Level=6, StaminaRaw=1124.13
Level=7, StaminaRaw=1191.53
Level=8, StaminaRaw=1259.80
Level=9, StaminaRaw=1328.97
Level=10, StaminaRaw=1399.05
Level=11, StaminaRaw=1470.04
Level=12, StaminaRaw=1541.95
Level=13, StaminaRaw=1614.81
Level=14, StaminaRaw=1688.63
Level=15, StaminaRaw=1763.40
Level=16, StaminaRaw=1839.16
Level=17, StaminaRaw=1915.91
Level=18, StaminaRaw=1993.66
Level=19, StaminaRaw=2072.43
Level=20, StaminaRaw=2152.22
Level=21, StaminaRaw=2233.07
Level=22, StaminaRaw=2314.97
Level=23, StaminaRaw=2397.94
Level=24, StaminaRaw=2481.99
Level=25, StaminaRaw=2567.15
Level=26, StaminaRaw=2653.42
Level=27, StaminaRaw=2740.81
Level=28, StaminaRaw=2829.36
Level=29, StaminaRaw=2919.05
Level=30, StaminaRaw=3009.93
Level=31, StaminaRaw=3101.99
Level=32, StaminaRaw=3195.25
Level=33, StaminaRaw=3289.74
Level=34, StaminaRaw=3385.46
Level=35, StaminaRaw=3482.4

,Level,Raw Stamina,Final Stamina
0,1,799.989280,800
1,2,863.143958,860
2,3,927.124699,930
3,4,991.942308,990
4,5,1057.607731,1060
5,6,1124.132058,1120
6,7,1191.526524,1190
7,8,1259.802509,1260
8,9,1328.971543,1330
9,10,1399.045309,1400
